# 論文 1：複雜動力學第一定律（The First Law of Complexodynamics）
## Scott Aaronson

### 實作：細胞自動機與熵增長

本筆記本展示封閉系統中複雜度（Complexity）與熵（Entropy）如何透過細胞自動機（Cellular Automata）增加。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import entropy

np.random.seed(42)

## 一維細胞自動機（規則 30 - 混沌型）

In [ ]:
def rule_30(left, center, right):
    """規則 30：產生複雜的混沌模式"""
    pattern = (left << 2) | (center << 1) | right
    rule = 30
    return (rule >> pattern) & 1

def evolve_ca(initial_state, steps, rule_func):
    """演化細胞自動機"""
    size = len(initial_state)
    history = np.zeros((steps, size), dtype=int)
    history[0] = initial_state
    
    for t in range(1, steps):
        for i in range(size):
            left = history[t-1, (i-1) % size]
            center = history[t-1, i]
            right = history[t-1, (i+1) % size]
            history[t, i] = rule_func(left, center, right)
    
    return history

# 簡單的初始狀態
size = 100
initial = np.zeros(size, dtype=int)
initial[size // 2] = 1  # 中央單一細胞

# 演化
steps = 100
evolution = evolve_ca(initial, steps, rule_30)

plt.figure(figsize=(12, 6))
plt.imshow(evolution, cmap='binary', interpolation='nearest')
plt.title('規則 30 細胞自動機 - 從簡單初始狀態產生的複雜度增長')
plt.xlabel('細胞位置')
plt.ylabel('時間步數')
plt.colorbar(label='狀態')
plt.show()

## 透過熵測量複雜度增長

In [ ]:
def measure_entropy_over_time(history):
    """測量每個時間步的夏農熵（Shannon Entropy）"""
    entropies = []
    
    for t in range(len(history)):
        state = history[t]
        # 計算機率分佈
        unique, counts = np.unique(state, return_counts=True)
        probs = counts / len(state)
        ent = entropy(probs, base=2)
        entropies.append(ent)
    
    return np.array(entropies)

def measure_spatial_complexity(history):
    """測量空間模式複雜度（轉換次數）"""
    complexities = []
    
    for t in range(len(history)):
        state = history[t]
        # 計算相鄰細胞之間的轉換次數
        transitions = np.sum(np.abs(np.diff(state)))
        complexities.append(transitions)
    
    return np.array(complexities)

entropies = measure_entropy_over_time(evolution)
complexities = measure_spatial_complexity(evolution)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))

ax1.plot(entropies, linewidth=2)
ax1.set_xlabel('時間步數')
ax1.set_ylabel('夏農熵（位元）')
ax1.set_title('熵隨時間增長')
ax1.grid(True, alpha=0.3)

ax2.plot(complexities, linewidth=2, color='orange')
ax2.set_xlabel('時間步數')
ax2.set_ylabel('空間複雜度（轉換次數）')
ax2.set_title('空間模式複雜度')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"初始熵：{entropies[0]:.4f} 位元")
print(f"最終熵：{entropies[-1]:.4f} 位元")
print(f"熵增加量：{entropies[-1] - entropies[0]:.4f} 位元")

## 咖啡自動機：不可逆混合

展示簡單初始狀態如何演化為複雜模式（就像咖啡中的奶油混合），但逆向過程是不太可能發生的。

In [ ]:
def diffusion_2d(grid, steps, diffusion_rate=0.1):
    """簡單的二維擴散模擬"""
    history = [grid.copy()]
    
    for _ in range(steps):
        new_grid = grid.copy()
        h, w = grid.shape
        
        for i in range(1, h-1):
            for j in range(1, w-1):
                # 與鄰居取平均
                neighbors = (
                    grid[i-1, j] + grid[i+1, j] + 
                    grid[i, j-1] + grid[i, j+1]
                ) / 4
                new_grid[i, j] = (
                    (1 - diffusion_rate) * grid[i, j] + 
                    diffusion_rate * neighbors
                )
        
        grid = new_grid
        history.append(grid.copy())
    
    return np.array(history)

# 建立初始狀態：咖啡中濃縮的「奶油」
size = 50
grid = np.zeros((size, size))
grid[20:30, 20:30] = 1.0  # 集中區域

# 模擬混合過程
mixing_history = diffusion_2d(grid, steps=50, diffusion_rate=0.2)

# 視覺化混合過程
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
timesteps = [0, 5, 10, 15, 20, 30, 40, 50]

for idx, (ax, t) in enumerate(zip(axes.flat, timesteps)):
    ax.imshow(mixing_history[t], cmap='YlOrBr', vmin=0, vmax=1)
    ax.set_title(f'時間步數 {t}')
    ax.axis('off')

plt.suptitle('不可逆混合：咖啡自動機', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

# 測量混合過程中的熵增長
mixing_entropies = []
for t in range(len(mixing_history)):
    flat = mixing_history[t].flatten()
    # 離散化以計算熵
    bins = np.histogram(flat, bins=20)[0]
    probs = bins[bins > 0] / bins.sum()
    mixing_entropies.append(entropy(probs, base=2))

plt.figure(figsize=(10, 5))
plt.plot(mixing_entropies, linewidth=2)
plt.xlabel('時間步數')
plt.ylabel('空間熵（位元）')
plt.title('混合過程中熵增加（熱力學第二定律）')
plt.grid(True, alpha=0.3)
plt.show()

print(f"\n關鍵洞察：簡單集中狀態 → 複雜混合狀態")
print(f"這個過程是不可逆的：你無法把咖啡和奶油分開！")

## 關鍵要點

1. **複雜度增長**：簡單初始狀態演化為複雜模式
2. **熵增加**：封閉系統趨向更高的熵（熱力學第二定律）
3. **不可逆性**：複雜狀態不太可能自發回到簡單狀態
4. **計算不可逆性**：咖啡自動機展示了根本性的限制

這與深度學習的關聯：
- 資訊理論的理解
- 學習表示的複雜度
- 損失函數和正則化中的熵